<a href="https://colab.research.google.com/github/rasim321/Bangla_GPT2/blob/master/Bangla_GPT2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tokenizers
!pip install transformers

     |████████████████████████████████| 3.3MB 8.0MB/s 
     |████████████████████████████████| 2.3MB 7.9MB/s 
     |████████████████████████████████| 901kB 51.0MB/s 


In [2]:
import os
from tokenizers.models import BPE
from tokenizers import Tokenizer
from tokenizers.decoders import ByteLevel as ByteLevelDecoder
from tokenizers.normalizers import NFKC, Sequence
from tokenizers.pre_tokenizers import ByteLevel
from tokenizers.trainers import BpeTrainer

class BPE_token(object):
    def __init__(self):
        self.tokenizer = Tokenizer(BPE())
        self.tokenizer.normalizer = NFKC()
        self.tokenizer.pre_tokenizer = ByteLevel()
        self.tokenizer.decoder = ByteLevelDecoder()

    def bpe_train(self, paths):
        trainer = BpeTrainer(vocab_size=50000, show_progress=True, inital_alphabet=ByteLevel.alphabet(), special_tokens=[
            "<s>",
            "<pad>",
            "</s>",
            "<unk>",
            "<mask>"
        ])
        self.tokenizer.train(paths, trainer)

    def save_tokenizer(self, location, prefix=None):
        if not os.path.exists(location):
            os.makedirs(location)
        self.tokenizer.model.save(location, prefix)


In [14]:
import glob
paths = glob.glob('/content/*.txt')

# for i in glob.glob('/content/*.txt'):
#   with open(i, 'r') as file:
#       paths.append(str(file.read().replace('\n', '')))


In [15]:
from pathlib import Path
import os

# the folder 'text' contains all the files
tokenizer = BPE_token()
# train the tokenizer model
tokenizer.bpe_train(glob.glob('/content/*.txt'))
# saving the tokenized data in our specified folder 
save_path = 'tokenized_data'
tokenizer.save_tokenizer(save_path)

In [16]:
import tensorflow as tf
from transformers import GPT2Config, TFGPT2LMHeadModel, GPT2Tokenizer
# loading tokenizer from the saved model path
tokenizer = GPT2Tokenizer.from_pretrained(save_path)
tokenizer.add_special_tokens({
  "eos_token": "</s>",
  "bos_token": "<s>",
  "unk_token": "<unk>",
  "pad_token": "<pad>",
  "mask_token": "<mask>"
})
# creating the configurations from which the model can be made
config = GPT2Config(
  vocab_size=tokenizer.vocab_size,
  bos_token_id=tokenizer.bos_token_id,
  eos_token_id=tokenizer.eos_token_id
)
# creating the model
model = TFGPT2LMHeadModel(config)

In [17]:
single_string = ''
for filename in paths:
  with open(filename, "r", encoding='utf-8') as f:
   x = f.read()
  single_string += x + tokenizer.eos_token
string_tokenized = tokenizer.encode(single_string)

In [18]:
examples = []
block_size = 100
BATCH_SIZE = 12
BUFFER_SIZE = 1000
for i in range(0, len(string_tokenized) - block_size + 1, block_size):
  examples.append(string_tokenized[i:i + block_size])
inputs, labels = [], []
for ex in examples:
  inputs.append(ex[:-1])
  labels.append(ex[1:])
dataset = tf.data.Dataset.from_tensor_slices((inputs, labels))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [24]:
learning_rate = 3e-5
epsilon = 1e-08
clipnorm = 1.0
epochs = 30

# defining our optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, epsilon=epsilon, clipnorm=clipnorm)
# definining our loss function
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# defining our metric which we want to observe
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
# compiling the model
model.compile(optimizer=optimizer, loss=[loss, *[None] * model.config.n_layer], metrics=[metric])

In [25]:
num_epoch = 10
history = model.fit(dataset, epochs=num_epoch)

Epoch 1/10
1270/1270 [==============================] - 359s 274ms/step - loss: 1.9283 - logits_loss: 1.9283 - logits_accuracy: 0.5381 - past_key_values_1_accuracy: 1.2574e-04 - past_key_values_2_accuracy: 1.6631e-04 - past_key_values_3_accuracy: 1.4767e-04 - past_key_values_4_accuracy: 1.6390e-04 - past_key_values_5_accuracy: 1.4642e-04 - past_key_values_6_accuracy: 1.3847e-04 - past_key_values_7_accuracy: 1.2822e-04 - past_key_values_8_accuracy: 1.3745e-04 - past_key_values_9_accuracy: 1.3643e-04 - past_key_values_10_accuracy: 1.5877e-04 - past_key_values_11_accuracy: 1.3433e-04 - past_key_values_12_accuracy: 1.4049e-04
Epoch 2/10
1270/1270 [==============================] - 348s 274ms/step - loss: 1.8721 - logits_loss: 1.8721 - logits_accuracy: 0.5477 - past_key_values_1_accuracy: 1.2345e-04 - past_key_values_2_accuracy: 1.6089e-04 - past_key_values_3_accuracy: 1.4093e-04 - past_key_values_4_accuracy: 1.7050e-04 - past_key_values_5_accuracy: 1.4938e-04 - past_key_values_6_accuracy: 

In [27]:
text = "মিসির আলির সব গল্প"
# encoding the input text
input_ids = tokenizer.encode(text, return_tensors='tf')
# getting out output
beam_output = model.generate(
  input_ids,
  max_length = 100,
  num_beams = 10,
  temperature = 0.7,
  no_repeat_ngram_size=2,
  num_return_sequences=5
)

print(tokenizer.decode(beam_output[0]))

Setting `pad_token_id` to 2 (first `eos_token_id`) to generate sequence


মিসির আলির সব গল্প শুনতে ভাল লাগছে। তিনি কিছুক্ষণের মধ্যেই তাকিয়ে আছেন।

মল্লিক সাহেব দীর্ঘ নিশ্বাস ফেলে তাঁর বিভ্ৰান্তিত মনে হলো। মি
